# Uncertainty Forest (UF) Demo
This demo provides the basic use cases of the `uncertainty_forest` module.

In [1]:
import numpy as np
from uncertainty_forest.uncertainty_forest import UncertaintyForest

## Hyperparameter Specification
Random forest hyperparameters such as minimum leaf size and maximum depth can be specified by the UF constructor.

In [9]:
# Notation from the current paper. 
max_depth = 30       # D
min_samples_leaf = 1 # k
max_features = None  # m
n_estimators = 200   # B
max_samples = .5     # s // 2
bootstrap = False    # Whether to subsample with replacement.
parallel = False

uf = UncertaintyForest(
    max_depth = max_depth,          
    min_samples_leaf = min_samples_leaf,    
    max_features = max_features,   
    n_estimators = n_estimators,   
    max_samples = max_samples,   
    bootstrap = bootstrap,
    parallel = parallel
)

In [10]:
# Or, you can just use the defaults.
uf = UncertaintyForest()

## Estimate the Conditional Probability of `Y` given `X = x`
Use `X_train` and `y_train` to estimate the posterior, and evaluate this posterior at `X_eval`.

In [11]:
n_class = 25
d = 10
classes = [-1, 0, 1]

X_train = np.concatenate([np.random.multivariate_normal(k*np.ones(d), np.eye(d), n_class) for k in classes])
y_train = np.concatenate([k*np.ones(n_class) for k in classes])
print(X_train.shape)
print(y_train.shape)

X_eval = np.array([c*np.ones(d) for c in range(-2, 3)])
print(X_eval)

(75, 10)
(75,)
[[-2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.]]


In [12]:
uf.fit(X_train, y_train)
cond_probability = uf.predict_proba(X_eval)
print("P(Y|X = x) for x in X_eval:")
print(cond_probability)

P(Y|X = x) for x in X_eval:
[[0.76038797 0.16347642 0.07613561]
 [0.71995983 0.20322769 0.07681249]
 [0.31995016 0.4704302  0.20961964]
 [0.09970353 0.22997999 0.67031649]
 [0.07898183 0.15495829 0.76605987]]


## Estimate the Conditional Entropy
`H(Y | X)` is also computed using the evaluation set `X_eval`.

In [13]:
cond_entropy = uf.estimate_cond_entropy(X_eval)
print("0 <= H(Y|X) = %f <= log2(3) = %f" % (cond_entropy, np.log2(3.0)))

0 <= H(Y|X) = 1.164090 <= log2(3) = 1.584963


# Estimate the mutual information
Mutual information can be estimated with the exact same protocol as conditional entropy, with `estimate_mutual_info`.

In [15]:
mutual_info = uf.estimate_mutual_info(X_eval)
print("I(X, Y) = %f" % mutual_info)

I(X, Y) = 0.420872
